In [ ]:
import datetime as dt

from ultralytics import YOLO, RTDETR

from ml_carbucks import YOLO_PRETRAINED_11L, YOLO_PRETRAINED_11N, DATA_CAR_DD_YAML, RESULTS_DIR

In [ ]:



# NOTE: Honestly I have no idea what are those YOLO11l and YOLO11n and why there are being duplicated, please explain guys 
# EDIT: I know now :)
# model = YOLO(YOLO_PRETRAINED_11L)
model = RTDETR("rtdetr-l.pt")

RUN_NAME = dt.datetime.now().strftime("%Y%m%d_%H%M%S") 

start_time = dt.datetime.now()
results = model.train(
    pretrained=True,         # use pretrained weights
    seed=42,                 # random seed for reproducibility
    data=DATA_CAR_DD_YAML,
    project=str(RESULTS_DIR),
    name=RUN_NAME,
    epochs=5,                # minimal epochs for speed
    imgsz=320,               # smaller image size for faster training
    batch=8,                 # small batch size to reduce memory and speed up
    workers=0,               # minimal data loading workers
    device='0',              # use GPU if available, 'cpu' for CPU
    optimizer='AdamW',       # optimizer type
    lr0=0.01,                # initial learning rate
    patience=5,              # early stopping
    verbose=False,           # less output
    # Additional useful params:
    # weight_decay=0.0005,     # weight decay for optimizer
    # momentum=0.937,          # momentum for SGD/AdamW    
    # box=7.5,                 # box loss gain
    # cls=0.5,                 # class loss gain
    # dfl=1.5,                 # dfl loss gain
    # hsv_h=0.015,             # image HSV-Hue augmentation
    # hsv_s=0.7,               # image HSV-Saturation augmentation
    # hsv_v=0.4,               # image HSV-Value augmentation
    # degrees=0.0,             # image rotation augmentation
    # translate=0.1,           # image translation augmentation
    # scale=0.5,               # image scale augmentation
    # shear=0.0,               # image shear augmentation
    # perspective=0.0,         # image perspective augmentation
    # flipud=0.0,              # image flip up-down
    # fliplr=0.5,              # image flip left-right
    # mosaic=1.0,              # mosaic augmentation probability
    # mixup=0.0,               # mixup augmentation probability
    # copy_paste=0.0,          # copy-paste augmentation probability
    # # ... see YOLO docs for more options
)

end_time = dt.datetime.now()

elapsed_time = end_time - start_time

print(f"Training completed in: {elapsed_time.total_seconds()}s")

In [ ]:
results.results_dict

In [ ]:
from pprint import pprint
pprint(results)


pprint(results.fitness)

In [ ]:
import base64

img = base64.b64encode(open('/home/bachelor/ml-carbucks/data/car_dd/images/train/000002.jpg', 'rb').read()).decode('utf-8')

print(img)

open('out.txt', 'w').write(img)

In [ ]:
from pathlib import Path
from typing import Any, Callable, Dict, Union

from optuna import Trial
import optuna


def get_trial_params(trial: Trial) -> Dict[str,Any]:
    epochs = trial.suggest_int('epochs', 10, 75)
    batch = trial.suggest_categorical('batch', [8, 16, 32, 64])
    imgsz = trial.suggest_categorical('imgsz', [320, 640, 960])
    lr = trial.suggest_float('lr', 1e-4, 1e-1, log=True)
    momentum = trial.suggest_float('momentum', 0.5, 0.99)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
    patience = trial.suggest_int('patience', 25, 50)


    return {
        'epochs': epochs,
        'batch': batch,
        'imgsz': imgsz,
        'lr0': lr,
        'momentum': momentum,
        'weight_decay': weight_decay,
        "patience": patience
    }

def create_objective(
    version: str,
    data: Path,
    name: str,
    device: str, 
)-> Callable:
    
    def objective(trial: Trial) -> float:
        params = get_trial_params(trial)
        model = YOLO(version)


        try:
            results = model.train(
                pretrained=True,
                seed=42,
                data=data,
                name=name,
                device=device,
                verbose=False,
                **params,
                save=False,
            )

            trial.set_user_attr("params", params)
            trial.set_user_attr("results", results.results_dict)

            return results.fitness[0]  # Return the primary fitness metric (e.g., mAP@50-95)

        except optuna.exceptions.TrialPruned as e:
            print("Trial pruned") # NOTE: this should be replace to logger
            raise e
        except Exception as e:
            print(f"Error in objective: {e}")
            raise e
        
    return objective

def execute_study(
    name: str,
    n_trials: int = 20,
    results_dir: Path = RESULTS_DIR,
    version: Union[Path,str] = YOLO_PRETRAINED_11N,
    data: Path = DATA_CAR_DD_YAML,
    direction: str = "maximize",
    device: str = '0',
):

    sql_path = results_dir / f"{name}.db"

    study = optuna.create_study(
        direction=direction,
        study_name=name,
        load_if_exists=True,
        storage=f"sqlite:///{sql_path}",
    )


    study.optimize(
        func=create_objective(
            version=str(version),
            data=data,
            name=name,
            device=device,
        ),
        n_trials=n_trials,
        gc_after_trial=True,
    )



# NOTE: This is how to execute hyperparameter optimization, but it takes a lot of time, so I commented it out for now

# execute_study(
#     name=f"{RUN_NAME}_optuna",
#     results_dir=RESULTS_DIR,
#     version=YOLO_PRETRAINED_11N,
#     data=DATA_CAR_DD_YAML,
#     direction="maximize",
#     device='0',
# )

# NOTE: to view optuna execute in terminal: optuna dashboard sqlite:///{sql_path}

     

In [ ]:
from ultralytics import YOLO
model = YOLO("/home/bachelor/ml-carbucks/results/large_1024_hyper&augm_v1/weights/best.pt")
image_dir = "/home/mainnural/shared"



In [ ]:
# Predict first 10 images in `image_dir`, export predictions to YOLO v1.1 format and create a zip archive
import os
from pathlib import Path
from typing import List
import shutil
import zipfile
import tempfile

# ...existing code above...
# Variables expected from earlier cells:
# - model: a loaded ultralytics.YOLO model
# - image_dir: path to directory with images

OUT_DIR = Path("predictions_yolo11")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# collect image paths (common image extensions)
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'}
image_dir_path = Path(image_dir)
images: List[Path] = sorted([p for p in image_dir_path.iterdir() if p.suffix.lower() in image_extensions])

if len(images) == 0:
    raise RuntimeError(f"No images found in {image_dir}")

images_to_process = images[:10]
print(f"Found {len(images)} images, processing {len(images_to_process)}")

# helper to convert boxes to YOLO v1.1 (normalized x_center,y_center,width,height)
def to_yolo_v11(box, img_w: int, img_h: int):
    # box: [x1, y1, x2, y2]
    x1, y1, x2, y2 = box
    w = x2 - x1
    h = y2 - y1
    x_center = x1 + w / 2.0
    y_center = y1 + h / 2.0
    return x_center / img_w, y_center / img_h, w / img_w, h / img_h


def _to_int(x):
    """Robustly convert various container/scalar types to int."""
    # handle ultralytics/torch/numpy containers
    try:
        if x is None:
            raise TypeError("None cannot be converted to int")
        # If x has .item()
        if hasattr(x, 'item'):
            return int(x.item())
        # If it's a list/tuple/numpy array with single element
        if isinstance(x, (list, tuple)) and len(x) == 1:
            return int(x[0])
        # If it's a sequence-like but has tolist()
        if hasattr(x, 'tolist'):
            t = x.tolist()
            if isinstance(t, (list, tuple)) and len(t) == 1:
                return int(t[0])
            if isinstance(t, (int, float)):
                return int(t)
        # last resort
        return int(x)
    except Exception:
        raise TypeError(f"Cannot convert {type(x)} to int")

# keep track of seen classes across predictions (so we can build obj.names if model.names not available)
classes_seen = set()

for img_path in images_to_process:
    print(f"Processing: {img_path.name}")
    # run prediction
    results = model.predict(source=str(img_path), imgsz=1024, conf=0.25, iou=0.45, device=model.device, verbose=False)

    # ultralytics returns a list of Results, one per image
    result = results[0]

    # get image size
    img_w, img_h = int(result.orig_img.shape[1]), int(result.orig_img.shape[0])

    # prepare output .txt file path (same base name as image)
    out_txt = OUT_DIR / (img_path.stem + ".txt")

    lines: List[str] = []

    # check if there are boxes
    if hasattr(result, 'boxes') and result.boxes is not None and len(result.boxes) > 0:
        for box in result.boxes:
            # box.xyxy, box.conf, box.cls
            # try to get a plain list [x1,y1,x2,y2] in all common shapes
            try:
                xyxy = box.xyxy.tolist()
            except Exception:
                try:
                    xyxy = list(box.xyxy)
                except Exception:
                    xyxy = None

            # handle nested/extra dimensions like [[x1,y1,x2,y2]] or array([[...]])
            if isinstance(xyxy, (list, tuple)) and len(xyxy) == 1 and isinstance(xyxy[0], (list, tuple)):
                xyxy = xyxy[0]

            # if still not a 4-tuple, try alternative access patterns
            if not (isinstance(xyxy, (list, tuple)) and len(xyxy) == 4):
                try:
                    # some ultralytics versions store .xyxy as tensor of shape (n,4) and indexing is needed
                    raw = getattr(box, 'xyxy', None)
                    if raw is not None:
                        maybe = raw[0]
                        try:
                            xyxy = maybe.tolist()
                        except Exception:
                            xyxy = list(maybe)
                except Exception:
                    pass

            if not (isinstance(xyxy, (list, tuple)) and len(xyxy) == 4):
                raise ValueError(f"Unexpected box.xyxy format for {img_path.name}: {xyxy}")

            # ensure floats
            xyxy = [float(v) for v in xyxy]

            raw_cls = getattr(box, 'cls', None)
            if raw_cls is None:
                # fallback to 0 if class is missing; change as needed
                cls = 0
            else:
                cls = _to_int(raw_cls)

            classes_seen.add(cls)

            x_center, y_center, bw, bh = to_yolo_v11(xyxy, img_w, img_h)
            # YOLO v1.1 format: class x_center y_center width height (normalized floats)
            lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")
    else:
        # No detections -> write empty file (YOLO expects either empty file or no file)
        lines = []

    out_txt.write_text("\n".join(lines))
    print(f"Wrote: {out_txt} ({len(lines)} boxes)")

print("Done exporting predictions to YOLO v1.1 format.")

# --- Build archive structure and zip it ---
# subset name for the annotations folder
subset = "predictions"
archive_name = OUT_DIR / "predictions_archive.zip"

# temp folder to build archive tree
with tempfile.TemporaryDirectory() as td:
    tmp = Path(td)
    # root of archive contents
    # create obj.names
    names_file = tmp / "obj.names"

    # try to get class names from model if available
    names_list = None
    if hasattr(model, 'names') and model.names:
        # model.names can be list or dict
        try:
            if isinstance(model.names, dict):
                # ensure ordered by index
                names_list = [model.names[i] for i in sorted(model.names.keys())]
            else:
                names_list = list(model.names)
        except Exception:
            names_list = None

    # if not available, create generic names based on seen classes
    if names_list is None:
        max_cls = max(classes_seen) if classes_seen else -1
        names_list = [f"class{i}" for i in range(max_cls + 1)] if max_cls >= 0 else ["class0"]

    names_file.write_text("\n".join(names_list))

    # create obj.data
    obj_data = tmp / "obj.data"
    num_classes = len(names_list)
    obj_data_contents = [f"classes = {num_classes}", f"train = train.txt", f"names = obj.names", f"backup = backup/"]
    obj_data.write_text("\n".join(obj_data_contents))

    # create annotations folder
    ann_dir = tmp / f"obj_{subset}_data"
    ann_dir.mkdir(parents=True, exist_ok=True)

    # copy annotation txts for processed images into ann_dir
    train_lines = []
    for img_path in images_to_process:
        src_txt = OUT_DIR / (img_path.stem + ".txt")
        if src_txt.exists():
            dst_txt = ann_dir / src_txt.name
            shutil.copy(src_txt, dst_txt)
        else:
            # create empty file if missing
            (ann_dir / (img_path.stem + ".txt")).write_text("")
        # write image path into train.txt as absolute path so downstream tools can find the images
        train_lines.append(str(img_path.resolve()))

    # write train.txt at root of archive
    (tmp / "train.txt").write_text("\n".join(train_lines))

    # create zip archive
    with zipfile.ZipFile(archive_name, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        # add obj.data and obj.names and train.txt
        zf.write(obj_data, arcname="obj.data")
        zf.write(names_file, arcname="obj.names")
        zf.write(tmp / "train.txt", arcname="train.txt")
        # add annotations folder files
        for f in sorted(ann_dir.iterdir()):
            zf.write(f, arcname=f"obj_{subset}_data/{f.name}")

print(f"Created archive: {archive_name}")
print("Archive contents:")
with zipfile.ZipFile(archive_name, 'r') as zf:
    for info in zf.infolist():
        print('-', info.filename)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import CocoDetection  # use torchvision, not effdet.data
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict
from effdet.efficientdet import HeadNet
from ml_carbucks import DATA_CAR_DD_DIR

# 1. Create config & model
model_name = "tf_efficientdet_d0"  # start with small model
config = get_efficientdet_config(model_name)
config.num_classes = 3  # your classes
config.image_size = (320, 320)  # or square int if config expects single int
# Optionally set other config fields:
# config.norm_kwargs = dict(eps=1e-3, momentum=0.01)

# 2. Build model
model = EfficientDet(config, pretrained_backbone=True)
model.class_net = HeadNet(config, num_outputs=config.num_classes)
# At this point, model and config should be consistent

# 3. Wrap for training
bench = DetBenchTrain(model).cuda()

# 4. Prepare dataset using torchvision
transform = transforms.Compose(
    [
        transforms.Resize(config.image_size),
        transforms.ToTensor(),
    ]
)
train_dataset = CocoDetection(
    root=DATA_CAR_DD_DIR / "images" / "train",
    annFile=str(DATA_CAR_DD_DIR / "instances_train.json"),
    transform=transform,
)


def coco_to_effdet_targets(coco_dataset):
    """
    Converts COCO annotations to effdet-compatible targets.
    Returns a list of dicts, one per image in coco_dataset.
    """
    # Group annotations by image_id
    targets = {
        "bbox": [],
        "cls": [],
    }

    for t in coco_dataset:
        bboxes = []
        labels = []
        for ann in t:
            x, y, w, h = ann["bbox"]
            bboxes.append([x, y, x + w, y + h])
            labels.append(ann["category_id"])

        if len(bboxes) == 0:
            targets["bbox"].append(torch.zeros((0, 4), dtype=torch.float32))
            targets["cls"].append(torch.zeros((0,), dtype=torch.int64))
        else:
            targets["bbox"].append(torch.tensor(bboxes, dtype=torch.float32))
            targets["cls"].append(torch.tensor(labels, dtype=torch.int64))
    return targets


def collate_fn(batch):
    imgs = torch.stack([b[0] for b in batch])
    targets = coco_to_effdet_targets([b[1] for b in batch])
    return imgs, targets


train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn,
)

# 5. Training loop
optimizer = torch.optim.AdamW(bench.parameters(), lr=1e-4)


for epoch in range(10):
    bench.train()
    for batch in train_loader:
        imgs, targets = batch

        imgs = imgs.cuda()

        # transform targets to move tensors to cuda
        targets = {
            "bbox": [t.cuda() for t in targets["bbox"]],
            "cls": [t.cuda() for t in targets["cls"]],
        }
        output = bench(imgs, targets)
        loss = output["loss"]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} loss: {loss.item()}")  # type: ignore


# 6. Save model weights (just the model)
torch.save(model.state_dict(), "efficientdet_d0_damage.pth")

# 7. Inference (wrap with DetBenchPredict)
predict_bench = DetBenchPredict(model).cuda()
predict_bench.eval()
# Example on one image:
# img_tensor = ToTensor()(Image.open(...)).unsqueeze(0).cuda()
# detections = predict_bench(img_tensor)


In [ ]:
model

In [ ]:
from typing import Optional
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont


def display_bbox(
    image_path: str,
    bbox: list,
    img_size: Optional[int] = None,
):
    # Load image
    img = Image.open(image_path)
    original_size = img.size
    if img_size is not None:
        # Resize while preserving aspect ratio
        img.thumbnail((img_size, img_size), Image.Resampling.LANCZOS) 
        new_img = Image.new("RGB", (img_size, img_size))
        new_img.paste(img, ((img_size - img.width) // 2, (img_size - img.height) // 2))

        img = new_img

    scale = img_size / max(original_size) if img_size else 1.0

     
    pad_x = 0
    pad_y = 0
    if img_size is not None:
        pad_x = (img_size - original_size[0] * scale) // 2
        pad_y = (img_size - original_size[1] * scale) // 2


    
    # Draw bbox

    draw = ImageDraw.Draw(img)
    x, y, w, h = bbox


    # rect = [x, y, x + w, y + h]
    rect = [
        x * scale + pad_x, 
        y * scale + pad_y, 
        pad_x + (x + w) * scale, 
        pad_y + (y + h) * scale
    ]

    draw.rectangle(rect, outline="red", width=3)

    # Optionally, draw area text

    try:
        font = ImageFont.truetype("arial.ttf", 24)
    except Exception:
        font = ImageFont.load_default()

    area = w * h
    area_text = f"Area: {area:.2f}"
    draw.text((x, y - 30), area_text, fill="red", font=font)

    # Display image
    img.show()

    

djson = {
    "id": 5010,
    "image_id": 2810,
    "category_id": 0,
    "bbox": [
        654.63,
        441.59985625,
        345.37,
        44.679995500000004
    ],
    "area": 15431.130045835002,
    "iscrowd": 0
}

pic_path = "/home/bachelor/ml-carbucks/data/car_dd/images/train/000021.jpg"

display_bbox(pic_path, djson['bbox'])
display_bbox(pic_path, djson['bbox'], img_size = 320)






In [ ]:
from typing import Optional, Tuple

import torch
from PIL import Image
from torchvision.transforms import ToTensor
import torch.nn.functional as F


def resize_with_padding_tensor(img_tensor: torch.Tensor, img_size: Optional[int] = None
                               ) -> Tuple[torch.Tensor, float, int, int]:
    """
    Efficiently resize [C,H,W] tensor to img_size x img_size with aspect ratio preserved,
    adding padding. Returns new tensor, scale, pad_x, pad_y.
    """
    C, H, W = img_tensor.shape
    if img_size is None:
        return img_tensor, 1.0, 0, 0

    scale = img_size / max(H, W)
    new_H, new_W = int(H * scale), int(W * scale)

    # Resize in a single step
    img_tensor = F.interpolate(img_tensor[None], size=(new_H, new_W), mode='bilinear', align_corners=False)[0]

    pad_x = (img_size - new_W) // 2
    pad_y = (img_size - new_H) // 2

    # Pad: (left, right, top, bottom)
    new_img = F.pad(img_tensor, (pad_x, img_size - new_W - pad_x, pad_y, img_size - new_H - pad_y))

    return new_img, scale, pad_x, pad_y

img_pil = Image.open(pic_path).convert("RGB")
img_tensor = ToTensor()(img_pil)

img_tensor_resized, scale, pad_x, pad_y = resize_with_padding_tensor(img_tensor, img_size=320)
print(f"Original size: {img_tensor.shape[1:]} -> Resized & padded size: {img_tensor_resized.shape[1:]}"
      f", scale: {scale:.4f}, pad_x: {pad_x}, pad_y: {pad_y}")
import matplotlib.pyplot as plt

def show_tensor_image(tensor, title=None):
    img = tensor.permute(1, 2, 0).cpu().numpy()
    plt.imshow(img)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()

show_tensor_image(img_tensor, title="Original Image Tensor")
show_tensor_image(img_tensor_resized, title="Resized & Padded Image Tensor")




In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
def ppp(tensor, bboxes_xyxy):
    img = tensor.cpu().permute(1, 2, 0).numpy()
    plt.imshow(img)
    for box in bboxes_xyxy:
        x1, y1, x2, y2 = box.cpu().numpy()
        rect = Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color='red', linewidth=2)
        plt.gca().add_patch(rect)
    plt.axis('off')
    plt.show() 
    
ppp(img_tensor_resized.cuda(), [[100,100,200,200]])

 